In [1]:
import json
import re
import os
import gzip
import numbers
import numpy as np
import pandas as pd
import multiprocessing as mp
from functools import partial
import statsmodels.api as sm
import seaborn as sns
import matplotlib as mpl
from IPython.display import display, HTML

spectra_map = {0: 'M', 1: 'K', 2: 'G', 3: 'F', 4: 'A', 5: 'B', 6: 'O', 7: 'X'}
singularity_flags = {
    'TidalLocked': 1,
    'TidalLocked2': 2,
    'TidalLocked4': 4,
    'LaySide': 8,
    'ClockwiseRotate': 16,
    'MultipleSatellites': 32,
}
star_type_map = {
    0: 'Main Sequence',
    1: 'Giant',
    2: 'White Dwarf',
    3: 'Neutron Star',
    4: 'Black Hole',
}
vein_type_map = {
    'Iron': 'Iron', 'Copper': 'Copper', 'Silicium': 'Silicon', 'Titanium': 'Titanium', 
    'Stone': 'Stone', 'Coal': 'Coal', 'Oil' : 'Oil', 'Fireice': 'Fire ice', 'Diamond': 'Kimberlite',
    'Fractal': 'Fractal Silicon', 'Crysrub': 'Organic Crystal', 'Grat': 'Optical Grating', 
    'Bamboo': 'Spiniform Stalagmite Crystal', 'Mag': 'Unipolar Magnets',
}
vein_types = list(vein_type_map.values())
ocean_types = ['Water', 'Sulfuric acid']
base_types = ['Iron', 'Copper', 'Silicon', 'Titanium', 'Stone', 'Coal']
gas_types = ['Hydrogen gas', 'Deuterium gas', 'Fire ice gas',]
S_scaled = gas_types + ['Oil']
M_scaled = [x for x in vein_types if x not in S_scaled]

previous_runs = [
    84459701, 16015752, 22778246, 26027001, 57786559, 51279269, 94520636, 
    32680784, 88834638, 22161804, 87759803, 76768927, 18238299,
]

%load_ext autotime

time: 191 µs (started: 2021-09-12 01:19:12 +01:00)


In [2]:
export_path = next(x for x in [
      os.path.expanduser('~/data/dsp/seeds'),
    '/mnt/c/' + r'\Users\Drake\Documents\Dyson Sphere Program\dsp_seedexporter'.replace('\\', '/'),
#     '/run/user/1000/gvfs/smb-share:server=chonyi.local,share=dsp_seedexporter/',
#     '/media/drake/Windows/Users/Drake/Documents/Dyson Sphere Program/dsp_seedexporter/',
] if os.path.isdir(x))
seed_re = re.compile(r'seed_(?P<seed>\d{8}).json.gz')

time: 2.68 ms (started: 2021-09-12 01:19:12 +01:00)


In [3]:
files = sorted(filter(seed_re.match, os.listdir(export_path)))
seeds = list(map(lambda x: seed_re.match(x).group('seed'), files))
print('{:d} files detected'.format(len(files)))

4558 files detected
time: 19.6 ms (started: 2021-09-12 01:19:12 +01:00)


In [4]:
def process_universe(func, file):
    try:
        with gzip.GzipFile(os.path.join(export_path, file), mode='r') as fd:
            universe = json.load(fd)
    except EOFError as e:
        raise ValueError('Bad file: ' + file)
    return func(universe)

def passthrough(x):
    return x

subset = ['seed_18238299.json.gz', 'seed_94520636.json.gz', 'seed_94520637.json.gz']
with mp.Pool(mp.cpu_count() - 1) as pool:
    universes = pool.map(partial(process_universe, passthrough), subset, chunksize=100)
universe = universes[0]

time: 194 ms (started: 2021-09-12 01:19:12 +01:00)


In [5]:
def check_files(file):
    try:
        with gzip.GzipFile(os.path.join(export_path, file), mode='r') as fd:
            universe = json.load(fd)
    except EOFError as e:
        return file

with mp.Pool(mp.cpu_count() - 1) as pool:
    bad_files = [x for x in pool.imap_unordered(check_files, files, chunksize=100) if x is not None]
if bad_files:
    raise IOError(bad_files)

time: 5.41 s (started: 2021-09-12 01:19:12 +01:00)


In [6]:
for bad_file in bad_files:
    os.remove(os.path.join(export_path, bad_file))

time: 266 µs (started: 2021-09-12 01:19:18 +01:00)


In [7]:
def distance(star):
    return sum(np.fromiter(star['position'].values(), dtype='float') ** 2) ** 0.5

distance_df = pd.DataFrame.from_records(
    data=[(star_name, distance(star)) for star_name, star in universe['star'].items()], 
    columns=['star_name', 'abs. dist.']
).sort_values('abs. dist.')
distance_df

,star_name,abs. dist.
0,Procyon,0.000000
1,Tramontana,2.960353
16,Zeta Arietis,4.005012
17,27 Ophiuchii,4.611032
4,Lambda Sagittae,5.189216
...,...,...
40,ζ Chamaeleontis,27.360977
45,Cymbae,30.245616
51,Theta Puppis,33.166251
60,Arided,36.442384


time: 17.2 ms (started: 2021-09-12 01:19:18 +01:00)


In [8]:
home_star = distance_df.iloc[0]
assert home_star['abs. dist.'] == 0.
home_system = universe['star'][home_star['star_name']]

time: 770 µs (started: 2021-09-12 01:19:18 +01:00)


In [9]:
def map_strings(universe):
    planet_types = []
    star_types = []
    singularities = []
    planet_count = 0
    for star in universe['star'].values():
        try:
            star_types.append((star['type'], star['typeString']))
        except KeyError:
            pass
        for planet in star['planet'].values():
            try:
                singularities.append((planet['singularity'], planet['singularityString']))
            except KeyError:
                pass
            try:
                planet_types.append((planet['type'], planet['typeString']))
            except KeyError:
                pass
            planet_count += 1
    return (
        np.unique(planet_types, axis=0), 
        np.unique(singularities, axis=0), 
        np.unique(star_types, axis=0), 
        planet_count
    )


with mp.Pool(mp.cpu_count() - 1) as pool:
    planet_types, singularities, star_types, planet_count = zip(*pool.imap_unordered(partial(process_universe, map_strings), files, chunksize=100))
planet_types = np.unique(np.concatenate([x for x in planet_types if len(x) > 0]), axis=0)
singularities = np.unique(np.concatenate([x for x in singularities if len(x) > 0]), axis=0)
star_types = np.unique(np.concatenate([x for x in star_types if len(x) > 0]), axis=0)


time: 5.66 s (started: 2021-09-12 01:19:18 +01:00)


In [10]:
display(pd.DataFrame.from_records(planet_types, columns=['code', 'type']))
display(pd.DataFrame.from_records(star_types, columns=['code', 'type']))

,code,type
0,1,Lava
1,1,Volcanic Ash
2,2,Mediterranean
3,2,Oceanic Jungle
4,2,Prairie
5,2,Red Stone
6,2,Sakura Ocean
7,2,Waterworld
8,3,Arid Desert
9,3,Ashen Gelisol


,code,type
0,0,A type star
1,0,B type star
2,0,F type star
3,0,G type star
4,0,K type star
5,0,M type star
6,0,O type star
7,1,Blue giant
8,1,Red giant
9,1,White giant


time: 9.3 ms (started: 2021-09-12 01:19:23 +01:00)


In [11]:
OIL_SPEED_MULTIPLIER = 4e-5
DYSON_SPHERE_BUFFER = 1.35

def reorder(df):
    resource_cols = vein_types + gas_types
    first_cols = [x for x in df.columns if x not in resource_cols]
    df[resource_cols] = df[resource_cols].fillna(0)
    return df[first_cols + resource_cols]

def scan_systems(universe):
    home_star_id = universe['meta']['birthStarId']
    home_planet_id = universe['meta']['birthPlanetId']
    seed = universe['meta']['seed']
    stars = []
    planets = []
    
    for star in universe['star'].values():
        max_dyson_rad = star['dysonRadius'] * 2

        star_dict = {
            'seed': seed,
            'star': star['name'],
            'home_system': star['id'] == home_star_id,
            'spectrum': spectra_map[star['spectr']],
            'type': star_type_map[star['type']],
            'luminosity': star['luminosity'],
        }
        stars.append(star_dict)

        for planet in star['planet'].values():
            planet_dict = {
                'seed': seed,
                'star': star['name'],
                'planet': planet['name'],
                'home_system': star['id'] == home_star_id,
                'homeworld': planet['id'] == home_planet_id,
                'type': planet.get('typeString', None),
                'tidal_lock': (planet['singularity'] & singularity_flags['TidalLocked']) > 0,
                'in_sphere': planet['sunDistance'] <= max_dyson_rad / DYSON_SPHERE_BUFFER,
                'ocean': planet.get('waterItem', None),
                
            }
            if 'vein' in planet:
                planet_veins = {vein_type_map[k]: v for k, v in planet['vein'].items()}
            elif 'gas' in planet:
                planet_veins = {(k + ' gas'): float(v['gasSpeed']) for k,v in planet['gas'].items()}
            else:
                raise ValueError('Bad planet: {}'.format(planet['name']))
            assert all(isinstance(x, (int, float)) for x in planet_veins.values()), planet_veins
            planets.append({**planet_dict, **planet_veins})
            
    planet_df = pd.DataFrame.from_records(planets)
    planet_df['Oil'] *= OIL_SPEED_MULTIPLIER
    
    star_df = pd.DataFrame.from_records(stars)
    systems = planet_df.groupby('star')
    star_resources = systems.sum().reindex(vein_types + gas_types, axis='columns')
    star_resources['planets'] = systems['planet'].count()
    oceans = (systems['ocean'].value_counts().unstack() > 0)[['Water', 'Sulfuric acid']]
    oceans.reindex(star_df['star'], fill_value=False)
    star_df = (star_df
               .merge(star_resources, left_on='star', right_index=True, validate='1:1')
               .merge(oceans.reindex(star_df['star'], fill_value=False), 
                      left_on='star', right_index=True, validate='1:1'))
    
    return star_df, planet_df

star_df, planet_df = scan_systems(universe)
display(star_df.head())
display(planet_df.head())

,seed,star,home_system,spectrum,type,luminosity,Iron,Copper,Silicon,Titanium,...,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas,planets,Water,Sulfuric acid
0,18238299,Procyon,True,G,Main Sequence,0.978408,23539812.0,33240354.0,13467962.0,13893383.0,...,0.0,0.0,0.0,0.0,0.250872,0.0,0.597154,4,True,False
1,18238299,Tramontana,False,M,Main Sequence,0.583942,14053173.0,28310788.0,3529581.0,37695658.0,...,0.0,2664609.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
2,18238299,Gamma Vulpeculae,False,K,Main Sequence,0.823254,36914259.0,41160402.0,20979566.0,37358159.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
3,18238299,YedPrior,False,M,Main Sequence,0.497194,26414756.0,18726224.0,19573996.0,39492924.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
4,18238299,Lambda Sagittae,False,K,Main Sequence,0.817663,9531719.0,1975948.0,4960219.0,31078570.0,...,0.0,958814.0,0.0,0.0,0.000000,0.0,0.000000,2,True,False


,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,ocean,Iron,...,Hydrogen gas,Oil,Fire ice,Fractal Silicon,Optical Grating,Spiniform Stalagmite Crystal,Kimberlite,Organic Crystal,Deuterium gas,Unipolar Magnets
0,18238299,Procyon,Procyon I,True,False,Lava,False,True,None,19746067.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18238299,Procyon,Procyon II,True,False,Gobi,False,False,None,892945.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18238299,Procyon,Procyon III,True,False,Ice Giant,False,False,None,NaN,...,0.250872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18238299,Procyon,Procyon IV,True,True,Mediterranean,False,False,Water,2900800.0,...,NaN,53.22388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238299,Tramontana,Tramontana I,False,False,Arid Desert,False,True,None,1682756.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


time: 37.5 ms (started: 2021-09-12 01:19:23 +01:00)


In [12]:
with mp.Pool(mp.cpu_count() - 1) as pool:
    star_dfs, planet_dfs = zip(*pool.imap_unordered(partial(process_universe, scan_systems), files, chunksize=100))
star_df = reorder(pd.concat(star_dfs))
planet_df = reorder(pd.concat(planet_dfs))

time: 15 s (started: 2021-09-12 01:19:23 +01:00)


In [13]:
home_systems = star_df.query('home_system == True').copy()
home_systems.head()

,seed,star,home_system,spectrum,type,luminosity,planets,Water,Sulfuric acid,Iron,...,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
0,22140824,Castula,True,G,Main Sequence,0.990787,4,True,False,7398412.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.890729,0.031340,0.000000
0,22161804,Gamma Scuti,True,G,Main Sequence,1.028369,4,True,False,25202964.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.277277,0.000000,0.601948
0,22211669,Gamma Cygni,True,G,Main Sequence,0.988809,4,True,False,24701628.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.896245,0.036877,0.000000
0,22218560,Propus,True,G,Main Sequence,1.045236,4,True,False,5460529.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.248357,0.000000,0.559244
0,22224945,Vespertilio,True,G,Main Sequence,0.980522,4,True,False,9441317.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.793410,0.037623,0.000000


time: 16.9 ms (started: 2021-09-12 01:19:38 +01:00)


In [14]:
home_system_planets = planet_df.query('home_system == True').copy()
home_system_planets.head()

,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,ocean,Iron,...,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
0,22140824,Castula,Castula I,True,False,Gobi,False,False,None,1299329.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0
1,22140824,Castula,Castula II,True,False,Gas Giant,False,False,None,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.890729,0.03134,0.0
2,22140824,Castula,Castula III,True,True,Mediterranean,False,False,Water,2497312.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0
3,22140824,Castula,Castula IV,True,False,Ice Field Gelisol,False,False,Water,3601771.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0
0,22161804,Gamma Scuti,Gamma Scuti I,True,False,Lava,False,True,None,21656630.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0


time: 19.4 ms (started: 2021-09-12 01:19:38 +01:00)


In [15]:
home_system_planets['TL|IS'] = home_system_planets['tidal_lock'] | home_system_planets['in_sphere']
home_system_planets['TL&IS'] = home_system_planets['tidal_lock'] & home_system_planets['in_sphere']
tl_is = home_system_planets.groupby(['seed', 'star'])[['TL|IS', 'TL&IS', 'tidal_lock', 'in_sphere']].sum()
pd.DataFrame({col: tl_is.reset_index().groupby(col)['star'].count()
              for col in tl_is.columns}).fillna(0).astype(int)

,TL|IS,TL&IS,tidal_lock,in_sphere
0,1868,4456,4348,1919
1,2633,102,210,2639
2,57,0,0,0


time: 17.9 ms (started: 2021-09-12 01:19:38 +01:00)


In [16]:
res_df = star_df.copy()
res_df[vein_types + gas_types] = res_df[vein_types + gas_types] > 0
res_probs = res_df.groupby(['spectrum', 'type'])[ocean_types
                                                 + vein_types 
                                                 + gas_types
                                                ].mean()
res = 'Organic Crystal'
res_probs.loc[res_probs[res] > 0].sort_values(res, ascending=False).style.format(precision=2)

,,Water,Sulfuric acid,Iron,Copper,Silicon,Titanium,Stone,Coal,Oil,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
spectrum,type,,,,,,,,,,,,,,,,,,,
F,Main Sequence,0.79,0.39,1.00,1.00,1.00,1.00,1.00,1.00,0.54,0.68,0.31,0.21,0.44,0.19,0.26,0.00,0.46,0.23,0.23
G,Main Sequence,0.85,0.29,1.00,1.00,1.00,0.99,1.00,1.00,0.59,0.69,0.30,0.21,0.42,0.19,0.28,0.00,0.46,0.23,0.23
K,Main Sequence,0.85,0.18,0.99,0.99,0.98,0.96,0.99,0.99,0.50,0.78,0.31,0.27,0.41,0.23,0.31,0.00,0.31,0.16,0.16
M,Main Sequence,0.88,0.10,0.99,0.99,0.98,0.97,0.99,0.99,0.43,0.89,0.46,0.50,0.35,0.42,0.37,0.00,0.30,0.16,0.15
A,Main Sequence,0.54,0.53,1.00,1.00,1.00,1.00,1.00,1.00,0.29,0.56,0.51,0.33,0.23,0.30,0.17,0.00,0.53,0.28,0.28
B,Main Sequence,0.43,0.71,1.00,1.00,1.00,1.00,1.00,1.00,0.11,0.61,0.32,0.20,0.09,0.18,0.05,0.00,0.65,0.36,0.36
O,Main Sequence,0.38,0.83,1.00,1.00,1.00,1.00,1.00,1.00,0.11,0.71,0.79,0.58,0.08,0.57,0.08,0.00,0.68,0.39,0.39
M,Giant,0.01,0.37,1.00,1.00,0.94,0.94,1.00,1.00,0.01,0.00,0.46,0.29,0.01,0.24,0.01,0.00,0.55,0.27,0.27


time: 129 ms (started: 2021-09-12 01:19:39 +01:00)


In [17]:
def S_scaler(f):
    return '{:.2f}/s'.format(f)

def M_scaler(f):
    if f < 10e6:
        return '{:.1f}M'.format(f / 1e6)
    else:
        return '{:.0f}M'.format(f / 1e6)

formatters = {
    'limit': M_scaler,
    **{col: S_scaler for col in S_scaled},
    **{col: M_scaler for col in M_scaled},
    **{col + ' (Start)': S_scaler for col in S_scaled},
    **{col + ' (Start)': M_scaler for col in M_scaled},
}

def shader(series, pct):
    pct_subset = pct.loc[series.index, series.name] ** (1/3)
    cmap = mpl.cm.get_cmap('coolwarm_r')
    rgb = [sns.set_hls_values(color=cmap(x), h=None, l=0.75, s=None) for x in pct_subset]
    return ['background: {};'.format(mpl.colors.rgb2hex(x)) for x in rgb]

def show(full_df, df):
    shade_cols = [x for x in df.columns if x in formatters.keys()]
    pct = full_df[shade_cols].rank(pct=True)
    display(HTML(
        df.style
        .apply(func=shader, pct=pct, axis='index', subset=shade_cols)
        .format(formatter=formatters).to_html()
    ))

time: 714 µs (started: 2021-09-12 01:19:39 +01:00)


In [18]:
home_resources = home_systems.set_index('seed')[base_types + ['Oil', 'Fire ice', 'Fire ice gas']]
home_resources['limit'] =  home_resources[base_types].min(axis='columns')

seed_res = (
    tl_is.reset_index(level='star')
    .merge(
        home_resources,
        on='seed', validate='1:1')
    .merge(
        star_df.groupby('seed')[ocean_types + vein_types + gas_types].sum(), 
        on='seed', validate='1:1', suffixes=[' (Start)', ''])
)

time: 31.5 ms (started: 2021-09-12 01:19:39 +01:00)


In [19]:
enough_fire_ice = (
    (seed_res['Fire ice (Start)'] > 1e6) 
    | ((seed_res['Fire ice (Start)'] > 0) & (seed_res['Fire ice gas (Start)'] > 0))
)

potential = (
    seed_res.loc[enough_fire_ice]
#     .query('`TL|IS` >= 1')
    .query('`tidal_lock` >= 1')
#     .query('limit > 6e6')
#     .query('`Organic Crystal` > 50e6')
    .sort_values('limit', ascending=False)
#     .sort_values('Organic Crystal', ascending=False)
)

pre_run = potential.index.isin(previous_runs)
show(potential, potential.loc[pre_run])
show(potential, potential.loc[~pre_run].head(25))


,star,TL|IS,TL&IS,tidal_lock,in_sphere,Iron (Start),Copper (Start),Silicon (Start),Titanium (Start),Stone (Start),Coal (Start),Oil (Start),Fire ice (Start),Fire ice gas (Start),limit,Water,Sulfuric acid,Iron,Copper,Silicon,Titanium,Stone,Coal,Oil,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
seed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26027001,Maasym,1,1,1,1,28M,23M,6.0M,27M,10M,9.2M,52.71/s,19M,0.65/s,6.0M,43,31,4672M,6191M,2957M,5649M,4318M,685M,2234.22/s,579M,83M,97M,58M,67M,67M,18M,15.96/s,0.33/s,19.52/s


,star,TL|IS,TL&IS,tidal_lock,in_sphere,Iron (Start),Copper (Start),Silicon (Start),Titanium (Start),Stone (Start),Coal (Start),Oil (Start),Fire ice (Start),Fire ice gas (Start),limit,Water,Sulfuric acid,Iron,Copper,Silicon,Titanium,Stone,Coal,Oil,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
seed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43378616,Alkaphrah,1,1,1,1,28M,21M,12M,18M,11M,9.3M,47.21/s,0.2M,0.62/s,9.3M,43,27,4725M,5994M,2824M,5600M,3998M,788M,2052.99/s,727M,132M,104M,71M,46M,54M,11M,24.42/s,0.72/s,16.47/s
16452863,Lambda Crateris,1,1,1,1,29M,23M,9.1M,15M,12M,9.8M,53.43/s,0.3M,0.58/s,9.1M,41,25,4078M,5103M,2167M,4715M,2741M,465M,2060.06/s,538M,184M,259M,48M,89M,82M,6.6M,26.11/s,0.82/s,14.93/s
84459702,Theemim,1,0,1,0,7.4M,7.0M,14M,17M,7.1M,11M,47.40/s,3.0M,0.65/s,7.0M,46,29,6018M,7125M,4117M,7553M,6425M,855M,2412.53/s,1388M,250M,373M,101M,161M,82M,20M,30.48/s,1.00/s,15.43/s
57418554,Haris,1,0,1,0,6.6M,11M,17M,16M,15M,9.9M,51.17/s,6.0M,0.00/s,6.6M,50,26,5703M,6938M,3314M,6841M,4253M,900M,2490.16/s,978M,94M,108M,82M,99M,55M,2.8M,19.93/s,0.61/s,12.95/s
68258881,NairalZaurak,1,0,1,0,28M,21M,5.9M,27M,7.7M,8.2M,55.34/s,9.0M,0.00/s,5.9M,43,27,6214M,8254M,3508M,6947M,5554M,975M,2548.44/s,660M,65M,146M,108M,82M,66M,3.1M,22.02/s,0.62/s,16.87/s
98245197,Epsilon Pyxidis,1,1,1,1,28M,21M,5.5M,27M,7.0M,10M,49.04/s,1.5M,0.00/s,5.5M,45,28,4608M,5563M,2585M,5741M,4439M,580M,2039.22/s,921M,123M,113M,75M,77M,44M,6.9M,25.00/s,0.82/s,12.43/s
53058500,40 Canum Venaticorum,1,1,1,1,29M,21M,5.1M,26M,8.5M,11M,53.22/s,1.5M,0.64/s,5.1M,45,24,4325M,5561M,2128M,5199M,3722M,644M,2314.63/s,808M,105M,149M,66M,64M,64M,5.0M,25.17/s,0.91/s,9.39/s
18352161,Zubenelakrab,1,1,1,1,27M,22M,4.5M,25M,9.5M,9.7M,47.36/s,4.4M,0.61/s,4.5M,43,22,3920M,4961M,2228M,4658M,3981M,535M,1914.90/s,689M,104M,67M,54M,107M,31M,1.3M,30.01/s,1.01/s,15.84/s
102802,Syrma,1,0,1,0,28M,24M,3.9M,25M,10M,11M,51.40/s,1.4M,0.58/s,3.9M,49,26,4501M,5071M,2377M,5498M,3332M,591M,2211.44/s,716M,164M,96M,77M,73M,50M,5.2M,29.17/s,0.93/s,17.61/s


time: 71.7 ms (started: 2021-09-12 01:19:39 +01:00)
